# Step 5: Add a deployment pipeline

<div class="alert alert-warning"> This notebook has been last tested on a SageMaker Studio JupyterLab instance using the <code>SageMaker Distribution Image 3.0.1</code> and with the SageMaker Python SDK version <code>2.245.0</code></div>

In previous step you implemented an automated data processing and model building pipeline. Each run of the pipeline produces a new version of the model. This notebook implements the automated model deployment step in our ML workflow.

You use the second part of the [SageMaker MLOps project template](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates-sm.html#sagemaker-projects-templates-git-code-pipeline) to create a ready-to use model deployment CI/CD pipeline.

The MLOps project template implements a CodePipeline pipeline for deployment of models in the SageMaker model registry to SageMaker endpoints for real-time inference. When a new model version is registered and approved in the model registry, the pipeline automatically initiates and performs a deployment.

||||
|---|---|---|
|1. |Experiment in a notebook ||
|2. |Scale with SageMaker AI processing jobs and SageMaker SDK ||
|3. |Operationalize with ML pipeline, model registry, and feature store ||
|4. |Add a model building CI/CD pipeline ||
|5. |Add a model deployment pipeline |**<<<< YOU ARE HERE**|
|6. |Add model and data monitoring ||

<div class="alert alert-info"> Make sure you using <code>Python 3</code> kernel in JupyterLab for this notebook.</div>

In [1]:
import boto3
import sagemaker 
from time import gmtime, strftime, sleep
from IPython.display import HTML

sagemaker.__version__

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Fetched defaults config from location: /home/sagemaker-user/.config/sagemaker/config.yaml


'2.239.0'

In [2]:
%store -r 

%store

try:
    initialized
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN 00-start-here notebook   ")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")

Stored variables and their in-db values:
baseline_s3_url                         -> 's3://sagemaker-us-east-1-906545278380/from-idea-t
bucket_name                             -> 'sagemaker-us-east-1-906545278380'
bucket_prefix                           -> 'from-idea-to-prod/xgboost'
dataset_feature_group_name              -> 'from-idea-to-prod-12-06-30-53'
dataset_file_local_path                 -> 'data/bank-additional/bank-additional-full.csv'
domain_id                               -> 'd-igloxuzrs3z2'
evaluation_s3_url                       -> 's3://sagemaker-us-east-1-906545278380/from-idea-t
experiment_name                         -> 'from-idea-to-prod-experiment-11-21-33-07'
feature_store_bucket_prefix             -> 'from-idea-to-prod/feature-store'
initialized                             -> True
input_s3_url                            -> 's3://sagemaker-us-east-1-906545278380/from-idea-t
mlflow_arn                              -> 'arn:aws:sagemaker:us-east-1:906545278380:mlflow

In [3]:
sm = boto3.client("sagemaker")

In [48]:
try:
    print(project_name)
    print(project_id)
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("You must complete the notebook 04-sagemaker-project")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++")

mlops-02-14-11-03-33
p-ca7phmcraepa


## Explore the project in the Studio UI

<div class="alert alert-info">Make sure you run the step 4 notebook and successfully provisioned the MLOps project and run the project pipeline at least once.</div>

Click on the links constructed by the next code cell to see the project and the model package in the Studio UI. You must see at least one version of the model registered in the model package group.

In [5]:
# Show the project link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/projects/{}/">the project</a> in the Studio UI</b>'.format(
            domain_id, region, project_name))
)

# Show the model package link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/models/registered-models/{}-{}/versions">the model package versions</a> in the Studio UI</b>'.format(
            domain_id, region, project_name, project_id))
)

In [20]:
# check that the project exists
project_data = sm.describe_project(ProjectName=project_name)

In [21]:
assert project_data['ProjectStatus'] == 'CreateCompleted', 'Project must be created at this point!'

In [24]:
# check that at least one model version is registered in the model registry
model_packages = sm.list_model_packages(
    ModelPackageGroupName=f'{project_name}-{project_id}',
    ModelApprovalStatus='PendingManualApproval')

In [25]:
assert len(model_packages['ModelPackageSummaryList']) > 0, 'You must have at least one model version in the status PendingManualApproval'

---

## Working with MLOps project for model deployment
The template provisions a GitHub repository with configuration files to specify the model deployment steps, AWS CloudFormation templates to define endpoints as infrastructure, and seed code for testing the endpoint.

This template provides the following resources:

1. A GitHub repository that contains seed code that deploys models to endpoints in staging and production environments
2. An AWS CodePipeline pipeline that has `source`, `build`, `deploy-to-staging`, and `deploy-to-production` steps. The `source` step points to the GitHub repository, and the `build` step gets the code from that repository and generates CloudFormation stacks to deploy. The `deploy-to-staging` and `deploy-to-production` steps deploy the CloudFormation stacks to their respective environments. There is a manual approval step between the staging and production build steps, so that a MLOps engineer must approve the model before it is deployed to production.
3. An Amazon EventBridge rule to launch a CodePipeline pipeline execution when a model package version is approved or rejected.
4. There is also a manual approval step after the placeholder unit tests. You can implement your own tests to replace the placeholders tests.

The project template also deploys an Amazon S3 bucket to store artifacts, including CodePipeline and CodeBuild artifacts, and any artifacts generated from the SageMaker pipeline runs.

The following diagram shows the architecture.

<img src="img/mlops-model-deploy.png" width="600"/>

You don't need to implement any configuration changes for the project. The model deployment pipeline works out of the box.
To start the model deployment pipeline, you must approve the model version in the model registry.

### Approve a model version
Approving a model version causes the MLOps project to launch the model deployment process. 

In the first step, the model deployment pipeline deploys the model version to a staging SageMaker real-time inference end-point.

You can approve the model version either in Studio in the Model registry or do it programmatically in the notebook. Let's do it programatically.

In [32]:
# list all model packages and select the latest one
model_packages = [
    p["ModelPackageSummaryList"] for p in
    sm.get_paginator('list_model_packages').paginate(
        ModelPackageGroupName=f'{project_name}-{project_id}',
        ModelApprovalStatus='PendingManualApproval',
        SortBy="CreationTime",
        SortOrder="Descending",
    )
][0]

In [33]:
if len(model_packages) == 0:
    raise Exception(f"No model package is found for {model_package_group_name} model package group. Run a model creation pipeline first.")

print(f"There are {len(model_packages)} model versions in the {model_package_group_name} model package group")
print(f"Approve the most recent model package:")

latest_model_package_arn = model_packages[0]["ModelPackageArn"]
print(latest_model_package_arn)

There are 2 model versions in the from-idea-to-prod-pipeline-model-12-06-30-22 model package group
Approve the most recent model package:
arn:aws:sagemaker:us-east-1:906545278380:model-package/mlops-02-14-11-03-33-p-ca7phmcraepa/2


The following statement sets the `ModelApprovalStatus` for the most recent model package in the model registry to `Approved`. The model package state change launches the EventBridge rule and the rule launches the CodePipeline CI/CD pipeline with model deployment.

In [34]:
model_package_update_response = sm.update_model_package(
    ModelPackageArn=latest_model_package_arn,
    ModelApprovalStatus="Approved",
)

Click on the link constructed by the following code cell to see the last model version in the model registry in the Studio UI changed the **Status** to `Approved`:

In [40]:
# Show the model package version link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/models/registered-models/{}-{}/versions/version-{}/overview">the model package version</a> in the Studio UI</b>'.format(
            domain_id, region, project_name, project_id, latest_model_package_arn.split('/')[-1]))
)

### Deployment pipeline execution
Upon approval of a model version in the code cell above, the model deployment CI/CD pipeline performs the following actions:
1. Create CloudFormation parameter configuration files with the staging and production parameters for CloudFormation templates for SageMaker endpoint
1. Create a SageMaker real-time inference endpoint with the name `<PROJECT-NAME>-staging`
1. Run the test script on the staging endpoint
1. Wait until the test result is manually approved in [AWS CodePipeline console](https://console.aws.amazon.com/codesuite/codepipeline)
1. Create a SageMaker endpoint with the name `<PROJECT-NAME>-prod` in the current account

Wait about 5-10 minutes until the pipeline finishes deployment of the staging endpoint. You can see the status of the endpoint in the Studio UI in **Deployments** > **Endpoints**:

In [41]:
# Show the inference endpoints link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/inference-experience/endpoints">inference endpoints</a> in the Studio UI</b>'.format(
            domain_id, region))
)

After the endpoint status changed from `Creating` to `InService`, the staging endpoint is fully operational. You can launch the model deployment process to the production stage by manually approving the **DeployStaging** stage of the CodePipeline pipeline. In the next section you approve the model deployment and launch the second stage of the deployment into a production endpoint.

### Deploy the model version to production

<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <p style=" text-align: center; margin: auto;">Wait until staging endpoint status changes to InService, then continue with the following code cells.
    </p>
</div>

In [49]:
while sm.describe_endpoint(EndpointName=f'{project_name}-staging')['EndpointStatus'] != 'InService':
    sleep(10)
    print(f'Waiting untile the endpoint {project_name}-staging is operational')

print(f'Endpoint {project_name}-staging is InService')

Endpoint mlops-02-14-11-03-33-staging is InService


The next code cell construct a CodePipeline approval link - click on the link to to open the CodePipeline console with the pipeline execution workflow.

In the **DeployStaging stage**, choose **Review** on the **ApproveDeployment** step. You might wait until `TestStaging` step completes with `Succeeded` status. 

<img src="img/deploy-staging-review.png" alt="approve deployment" width="800px">

In the **Review** dialog box, select **Approve** and choose **Submit**.

In [57]:
# Show the approval link
display(
    HTML(
        '<b>Please approve the manual step in <a target="top" href="https://console.aws.amazon.com/codesuite/codepipeline/pipelines/sagemaker-{}-{}-modeldeploy/view?region={}">AWS CodePipeline</a></b>'.format(
            project_name, project_id, region)
    )
)

Approving the **DeployStaging** stage causes the deployment pipeline to continue and to deploy the model to the production endpoint. To view the endpoints, choose the **Deployments** > **Endpoints** in Studio UI or click on the link below.

In [58]:
# Show the inference endpoints link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/inference-experience/endpoints">inference endpoints</a> in the Studio UI</b>'.format(
            domain_id, region))
)

As your CI/CD deployment pipeline continues, you see the production endpoint in status `Creating` along with the previously deployed staging endpoint in status `InService`.

After 5-10 min the deployment is completed and both endpoints are in status `InService`.

Click on the link constructed by the following cell to see the endpoints that belongs to the project.
You see that both endpoints, `staging` and `prod`, are visible in the deployment project because the project and the endpoints are connected via the metadata and resource tags.

In [59]:
# Show the project link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/projects/{}/endpoints">endpoints of the project {}</a> in the Studio UI</b>'.format(
            domain_id, region, project_name, project_name))
)

## Summary
In this notebook you implemented an automated CI/CD deployment pipeline with the following features:
- use CloudFormation templates for SageMaker real-time inference endpoint deployment
- model approval in the model registry launches the model deployment pipeline
- model deployment pipeline contains two stages, staging and production with automated tests for the staging endpoint and manual approval for the production deployment, and final deployment of the production endpoint.

---

## Clean-up
<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <p style=" text-align: center; margin: auto;">
    If you're going to run the step 6 notebook (Data and Model Quality Monitoring), you need to keep at least one of the endpoints. If you finish the workshop here and don't run the step 6 notebook, navigate to the <b>clean-up notebook (99-clean-up.ipynb)</b> and follow the clean-up instructions to avoid charges in your AWS account.
    <br>
    <br>
    You don't need to run the clean-up if you're using an AWS-provided AWS account.
    </p>
</div>

## Continue with the step 6
open the step 6 [notebook](06-monitoring.ipynb).

## Further development ideas for your real-world projects
- Add end-to-end data encryption using AWS KMS keys
- Create a [custom SageMaker project template](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates-custom.html) for model deployment to cover your specific project requirements
- Add [multi-account model deployment](https://aws.amazon.com/blogs/machine-learning/multi-account-model-deployment-with-amazon-sagemaker-pipelines/) to your ML workflow
- Add automated model tests to the placeholder in the CodePipeline pipeline
- Use [Amazon SageMaker Inference Recommender](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-recommender.html) to run automated load tests for your inference endpoints and to select the best instance type and configuration

## Additional resources
- [Deploy a Machine Learning Model to a Real-Time Inference Endpoint](https://aws.amazon.com/getting-started/hands-on/machine-learning-tutorial-deploy-model-to-real-time-inference-endpoint/)
- [SageMaker MLOps Project Walkthrough](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough.html)
- [Amazon SageMaker Pipelines lab in SageMaker Immersion Day](https://catalog.us-east-1.prod.workshops.aws/workshops/63069e26-921c-4ce1-9cc7-dd882ff62575/en-US/lab6)
- [Amazon SageMaker secure MLOps](https://github.com/aws-samples/amazon-sagemaker-secure-mlops)
- [Testing approaches for Amazon SageMaker ML models](https://aws.amazon.com/blogs/machine-learning/testing-approaches-for-amazon-sagemaker-ml-models/)
- [Model hosting patterns in Amazon SageMaker blog series](https://aws.amazon.com/blogs/machine-learning/model-hosting-patterns-in-amazon-sagemaker-part-1-common-design-patterns-for-building-ml-applications-on-amazon-sagemaker/)
- [Take advantage of advanced deployment strategies using Amazon SageMaker deployment guardrails](https://aws.amazon.com/blogs/machine-learning/take-advantage-of-advanced-deployment-strategies-using-amazon-sagemaker-deployment-guardrails/)
- [MLOps deployment best practices for real-time inference model serving endpoints with Amazon SageMaker](https://aws.amazon.com/blogs/machine-learning/mlops-deployment-best-practices-for-real-time-inference-model-serving-endpoints-with-amazon-sagemaker/)

# Shutdown kernel

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>